In [ ]:
import feast
import pandas as pd
from datetime import datetime
import yaml

In [ ]:
with open('feature_repo/feature_store.yaml', 'r') as file:
    fs_config_yaml = yaml.safe_load(file)

In [ ]:
fs_config = feast.repo_config.RepoConfig(**fs_config_yaml)

In [ ]:
fs = feast.FeatureStore(config=fs_config)

In [ ]:
features = fs.get_online_features(
    features=[
        "transaction_features:distance_from_last_transaction",
        "transaction_features:ratio_to_median_purchase_price",
        "transaction_features:used_chip",
        "transaction_features:used_pin_number",
        "transaction_features:online_order",
    ],
    entity_rows=[
        {
            "transaction_id": 348007,
        }
    ],
)

In [ ]:
features = features.to_dict()
features

In [ ]:
pd.read_parquet('card_transaction_data.parquet', engine='pyarrow')

In [ ]:
deployed_model_name = "fraud"
infer_endpoint = ""
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
import requests


def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "input",
                "shape": [1, 5],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
data = [features["distance_from_last_transaction"][0], features["ratio_to_median_purchase_price"][0], features["used_chip"][0], features["used_pin_number"][0], features["online_order"][0]]
prediction = rest_request(data)
prediction